In [1]:
from scipy.ndimage import affine_transform

from aidream_registration import constants
import aidream_registration.utils.cohort_utils as cu
from aidream_registration.dataloaders import AtlasImagingNiftiLoader


In [2]:
from pathlib import Path
import shutil
import numpy as np
import os

import ants


In [8]:
# the list of perfusion patients :
list_patients = cu.get_perfusion_patients()

# the referential table :
df_ref = cu.get_referential_table(list_patients)


In [10]:
dir_freesurfer_results = Path("/media/maichi/Backup1") / "AIDREAM DATA" / "FREESURFER T1 SEGMENTATION"
assert dir_freesurfer_results.exists(), f"Directory {dir_freesurfer_results} does not exist"


In [11]:
for patient in list_patients:

    dir_patient = dir_freesurfer_results / patient
    dir_aseg = dir_patient / "mri" / "aseg.mgz"
    dir_aseg_auto = dir_patient / "mri" / "aseg.auto.mgz"

    if dir_aseg.exists():
        df_ref.loc[df_ref["AIDREAM_ID"] == patient, "FREESURFER"] = "ASEG"

    elif dir_aseg_auto.exists():
        df_ref.loc[df_ref["AIDREAM_ID"] == patient, "FREESURFER"] = "ASEG.AUTO"

    elif dir_patient.exists():
        df_ref.loc[df_ref["AIDREAM_ID"] == patient, "FREESURFER"] = "NOT DONE"

    else:
        df_ref.loc[df_ref["AIDREAM_ID"] == patient, "FREESURFER"] = "ERROR"






In [13]:
df_ref.groupby("FREESURFER").count()

,AIDREAM_ID,cohort_ID,local_ID,perfusion,complete_validation,surgery_type,pre_RT_reference,pre_RT_TPS,valid_pre_RT_segmentation_model,valid_pre_RT_perfusion_model,Rechute_reference,Rechute_TPS,valid_Rechute_segmentation_model,valid_Rechute_perfusion_model
FREESURFER,,,,,,,,,,,,,,
ASEG,152,152,152,152,152,152,152,152,152,152,149,152,152,152
ASEG.AUTO,12,12,12,12,12,12,12,12,12,12,12,12,12,12
ERROR,21,21,21,21,21,21,21,21,21,21,18,21,21,21
NOT DONE,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [23]:
df_ref.loc[df_ref["FREESURFER"] == "NOT DONE"]

,AIDREAM_ID,cohort_ID,local_ID,perfusion,complete_validation,surgery_type,pre_RT_reference,pre_RT_TPS,valid_pre_RT_segmentation_model,valid_pre_RT_perfusion_model,Rechute_reference,Rechute_TPS,valid_Rechute_segmentation_model,valid_Rechute_perfusion_model,FREESURFER
106,AIDREAM_345,new400pat,201919513NU,yes,1,2.0,GE_repNativAXIALISEDnew,GE,yes,yes,GE_repNativAXIALISEDnew,GE,yes,yes,NOT DONE


In [17]:
df_ref.to_excel("freesurfer_status.xlsx", index=False)


In [19]:
df_ref.loc[df_ref["FREESURFER"] == "ERROR"]

,AIDREAM_ID,cohort_ID,local_ID,perfusion,complete_validation,surgery_type,pre_RT_reference,pre_RT_TPS,valid_pre_RT_segmentation_model,valid_pre_RT_perfusion_model,Rechute_reference,Rechute_TPS,valid_Rechute_segmentation_model,valid_Rechute_perfusion_model,FREESURFER
73,AIDREAM_245,MMI-PROB_154,201305042UL,yes,1,0.0,AC_repATLAS,RS,yes,yes,AC_repATLAS,RS,yes,yes,ERROR
74,AIDREAM_248,new400pat,201806728SP,yes,1,1.0,GE_repNATIV,RS,yes,yes,Repère natif,RS,yes,yes,ERROR
78,AIDREAM_301,new400pat,202009913GU,yes,1,2.0,GE_repNATIV,RS,yes,yes,GE_repNativAXIALISEDnew,GE,yes,yes,ERROR
79,AIDREAM_306,new400pat,201317056FU,yes,1,1.0,GE_repNativAXIALISEDnew,GE,yes,yes,NaN,RS,yes,yes,ERROR
80,AIDREAM_307,new400pat,201501780EE,yes,1,1.0,GE_repNativAXIALISEDnew,GE,yes,yes,NaN,RS,yes,yes,ERROR
81,AIDREAM_308,new400pat,201508980RE,yes,1,2.0,GE_repNATIV,RS,yes,yes,NaN,RS,yes,yes,ERROR
83,AIDREAM_310,new400pat,201511878RF,yes,1,2.0,GE_repNativnew,GE,yes,yes,GE_repNativAXIALISEDnew,GE,yes,yes,ERROR
84,AIDREAM_311,new400pat,201514528KM,yes,1,1.0,GE_repNativnew,GE,yes,yes,GE_repNativAXIALISEDnew,GE,yes,yes,ERROR
85,AIDREAM_312,new400pat,201518647SW,yes,1,0.0,GE_repNATIV,RS,yes,yes,GE_repNativAXIALISEDnew,GE,yes,yes,ERROR
86,AIDREAM_314,new400pat,201515720DG,yes,1,0.0,GE_repNativnew,GE,yes,yes,GE_repNativAXIALISEDnew,GE,yes,yes,ERROR


In [20]:
with open("bad_patients.txt", "w") as f:
    for patient in df_ref.loc[df_ref["FREESURFER"] == "ERROR", "AIDREAM_ID"]:
        f.write(f"{patient}\n")

In [22]:
f.close()